In [3]:
import numpy as np

In [4]:
m=2
n=5

S = np.random.rand(m,1)
A = np.random.rand(m,m)
A, _ = np.linalg.qr(A) #ortogonalna da nam norme vektora ne eksplodiraju

for i in range(n):
    S = np.hstack((S, A@(S[:,-1].reshape(-1,1))))

S_bez_suma = S
S = S+np.random.normal(0, 0.01, S.shape)
X = S[:, :-1]
Y = S[:, 1:]

In [5]:
print(A)

[[-0.73577025 -0.67723123]
 [-0.67723123  0.73577025]]


In [6]:
print(X.T)

[[ 0.51372105  0.10172896]
 [-0.42823018 -0.25090966]
 [ 0.51840412  0.11729594]
 [-0.44115868 -0.24796666]
 [ 0.51094701  0.10603081]]


In [7]:
def stlnb_dmd(Xt, yi, i, w=100, tol=1e-5, max_iter=500):
    (n, m) = Xt.shape 
    yi = yi.reshape(n,1) #yi treba imati shape (n, 1), ovo ako slucajno ude kao (n,)
    E = np.zeros((n,m))
    alpha = np.zeros((m*n, 1))
    beta = 0.0

    ai,_,_,_ = np.linalg.lstsq(Xt, yi)
    print(ai.T)
    X = np.kron(np.eye(n), ai.reshape(-1))
    rh = yi - Xt@ai
    #print(rh)
    ai_norm = 1
    alpha_norm=1
    beta_norm=1
    rh_norm=1
    br=0

    P0 = np.zeros((n, m*n))
    for k in range(n-1):
        P0[k, (k+1)*m+i] = 1.0
    #print(P0)
    #print(P0) #dobar

    D1 = np.hstack((np.eye(m*n), np.zeros((m*n, 1+m))))
    for k in range(1,n):
        D1[k*m+i, k*m+i] = np.sqrt(2.0)
    D2 = np.zeros((1, m*n+m+1))
    D2[0,m*n] = 1.0
    print("D1 shape: {}, D2 shape: {}".format(D1.shape, D2.shape)) 

    print(P0)

    while((ai_norm>tol or alpha_norm>tol or beta_norm>tol) and br<max_iter and rh_norm>tol):
        if(rh_norm <tol):
            print("AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA")
        #print(X.shape, P0.shape, Xt.shape)
        M = np.hstack((w*(X-P0), -w*np.eye(n, 1, -n+1), w*(Xt+E)))
        M= np.vstack((M, D1, D2))

        print("M : ", M)
        print("E : ", E)
        print("[E|r] : ", np.hstack((E, beta*np.eye(n,1,-n+1)+P0@alpha)))
        #print(M)
        #return M
        #print(D1[:, :m*n]) #D1 je dobar
        rhs = np.vstack((-w*rh, D1[:, :m*n]@alpha, beta))
        sol,_,_,_ = np.linalg.lstsq(M, rhs)
        
        alpha += sol[:m*n, :]
        beta += sol[m*n]
        ai += sol[-m:, :]

        E = alpha.reshape(n,m) #koji redoslijed... trebalo bi biti dobro i konzistentno s P0 i ostalime..
        X = np.kron(np.eye(n), ai.reshape(-1))

        rh = -(yi+beta*np.eye(n,1,-n+1)+P0@alpha - (Xt+E)@ai)
        rh_norm = np.linalg.norm(rh)
        print("rh norm: {}".format(rh_norm))
        #print(rh)
        
        ai_norm = np.linalg.norm(ai)
        alpha_norm = np.linalg.norm(alpha)
        beta_norm = np.abs(beta)
        print(ai.T)
        print("ai norm: {}, alpha norm: {}, beta norm: {}".format(ai_norm, alpha_norm, beta_norm))
        #print(np.hstack((E, beta*np.eye(n,1,-n+1)+P0@alpha))) #dobar oblik
        
        br+=1

    return ai, np.hstack((E, beta*np.eye(n,1,-n+1)+P0@alpha))

In [8]:
ai, Er = stlnb_dmd(X.T, Y[1,:], 1,w=10000, tol=1e-5, max_iter=100)

[[-0.61979384  0.63141432]]
D1 shape: (10, 13), D2 shape: (1, 13)
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
M :  [[-6.19793845e+03  6.31414323e+03 -0.00000000e+00 -1.00000000e+04
  -0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
  -0.00000000e+00  0.00000000e+00 -0.00000000e+00  5.13721053e+03
   1.01728959e+03]
 [-0.00000000e+00  0.00000000e+00 -6.19793845e+03  6.31414323e+03
  -0.00000000e+00 -1.00000000e+04 -0.00000000e+00  0.00000000e+00
  -0.00000000e+00  0.00000000e+00 -0.00000000e+00 -4.28230183e+03
  -2.50909656e+03]
 [-0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
  -6.19793845e+03  6.31414323e+03 -0.00000000e+00 -1.00000000e+04
  -0.00000000e+00  0.00000000e+00 -0.00000000e+00  5.18404121e+03
   1.17295937e+03]
 [-0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
  -0.00000000e+00  0.00000000e+00 -6.19793845e

In [11]:
X.T-Er[:,:-1]

array([[ 0.51571625,  0.09969286],
       [-0.42269773, -0.25215278],
       [ 0.51604894,  0.12288834],
       [-0.44944564, -0.24563579],
       [ 0.50727102,  0.10129298]])

In [13]:
print(S_bez_suma[:,:-1].T)

print(S[:,:-1].T)

[[ 0.49736215  0.11095534]
 [-0.44108669 -0.25519154]
 [ 0.49736215  0.11095534]
 [-0.44108669 -0.25519154]
 [ 0.49736215  0.11095534]]
[[ 0.51372105  0.10172896]
 [-0.42823018 -0.25090966]
 [ 0.51840412  0.11729594]
 [-0.44115868 -0.24796666]
 [ 0.51094701  0.10603081]]


In [12]:
Er

array([[-0.0019952 ,  0.0020361 ,  0.00124312],
       [-0.00553246,  0.00124312, -0.00559241],
       [ 0.00235518, -0.00559241, -0.00233087],
       [ 0.00828696, -0.00233087,  0.00473783],
       [ 0.00367599,  0.00473783,  0.00587105]])

In [24]:
print((X.T-Er[:,:-1])-S[:,:-1].T)
print((Y[1,:])-Er[:,-1]-S[1,1:])

[[ 0.0019952  -0.0020361 ]
 [ 0.00553246 -0.00124312]
 [-0.00235518  0.00559241]
 [-0.00828696  0.00233087]
 [-0.00367599 -0.00473783]]
[-0.00124312  0.00559241  0.00233087 -0.00473783 -0.00587105]
